In [1]:
%%html
<style>
table {
    display: inline-block
}
</style>

# Docker应用：Centos操作系统
    在Docker的所有镜像中，操作系统镜像往往是作为应用的基础镜像而使用的，是项目制作镜像的基础。本章更多的是从操作系统本身特性入手来讲解Docker如何使用。因此需要有一定的Linux操作系统基础知识。

## 提纲

1. [概述](#1.概述)
2. [快速开始](#2.快速开始)
3. [构建(systemd_base_image)](#3.构建(systemd_base_image))

## 参考资料

## 资源

名称 | 说明 | 链接
:-- | :-- | :--
官网 | - | https://centos.org/
docker | - | https://hub.docker.com/_/centos
Dockerfile | docker-hub | https://github.com/docker-library/official-images/blob/master/library/centos
Dockerfile | 官方 | https://github.com/CentOS/CentOS-Dockerfiles
软件更新源 | 清华大学 | https://mirrors.tuna.tsinghua.edu.cn/help/centos/

# 1.概述
    CentOS是免费Linux中可以用于商用环境最为稳定可靠的操作系统，而且使用方便容易上手，在高职教育中普遍使用CentOS作为Linux教学内容。
    不过从镜像大小、兼容性、社区活跃程度、版本升级频度等综合考量，却很少作为应用的基础镜像；除此之外全球主流的云服务商提供的虚拟云主机往往很少有提供CentOS的虚拟主机。(目前活跃程度高、社区支持力度大的主流操作系统镜像有：Debian、Alpine和Ubuntu。)
    不过从学生熟悉的操作系统入手学习Docker系统镜像还是有很大帮助的。

# 2.快速开始

### 镜像下载
```shell
docker pull centos:7
docker pull ubuntu
docker pull alpine
docker pull debian
docker pull busybox
------------------------------
...

docker images
------------------------------
REPOSITORY                                TAG             IMAGE ID       CREATED         SIZE
debian                                    latest          e15dbfac2d2b   2 days ago      117MB
ubuntu                                    latest          ca2b0f26964c   6 weeks ago     77.9MB
alpine                                    latest          05455a08881e   2 months ago    7.38MB
busybox                                   latest          ba5dc23f65d4   10 months ago   4.26MB
centos                                    7               eeb6ee3f44bd   2 years ago     204MB
```
> 目前CentOS最新稳定版本为：7.9.2009。
<br> 为了比较，下载了多个流行的镜像。可以看出CentOS的镜像尺寸较大，升级非常缓慢，最新镜像也是1年多前的产品。

### 镜像运行
    
```shell
# 运行
docker run --rm -it -d --name centos7 --hostname centos centos:7
27cc2f367152ceb2c04d718d9f02277f065a8b501b535f72b7c1c2482fa1cdbe

# 进入容器 shell
docker exec -it centos7 bash
[root@centos /]#
```
> 当出现Linux命令提示符号 `#` 的时候，说明你已经进入了容器的隔离空间，并可以使用shell指令在容器内部进程中执行。
<br>课本P111～125，关于容器的主要命令。
<br>书中没有涉及的参见：`docker run --help`

### 操作系统CentOS的使用
```shell
yum makecache
---------------------------
Loaded plugins: fastestmirror, ovl
Determining fastest mirrors
 * base: mirrors.jlu.edu.cn
 * extras: mirrors.nju.edu.cn
 * updates: mirrors.nju.edu.cn
base                                                                                             | 3.6 kB  00:00:00
...中间省略...
Metadata Cache Created

yum install curl
---------------------------
...中间省略...
Updated:
  curl.x86_64 0:7.29.0-59.el7_9.2

Dependency Updated:
  libcurl.x86_64 0:7.29.0-59.el7_9.2

Complete!

curl baidu.com
---------------------------
<html>
<meta http-equiv="refresh" content="0;url=http://www.baidu.com/">
</html>
```
> 进入容器内部 shell 常用命令参考过去的Linux学习内容，看看你是否还熟悉？

## 整个操作过程参考如下
> <a href="https://asciinema.xtalker.cn:8000/a/6" target="_blank"><img style="width: 80%;display: inline-block"  src="https://asciinema.xtalker.cn:8000/a/6.svg" /></a>

# 3.构建(systemd_base_image)
    CentOS的官方镜像使用systemd来管理服务的启停，为此需要映射宿主机操作系统的 cgroups 环境。

## 3.1.官方建议
### systemd是什么
    systemd是Linux系统下的一个系统和服务管理器，它是Linux系统基础组件的集合，也是Linux系统第一个运行的进程，用来创建系统的守护进程以管理整个Linux系统。
    我们常用的 `systemctl` 指令就是systemd的客户端管理程序。
    
### Dockerfile
    创建一个目录 centos 用于存放我们的代码文件
    创建一个文本文件：“Dockerfile”，注意没有扩展名，其内容如下：
```shell
FROM centos:7
ENV container docker
RUN (cd /lib/systemd/system/sysinit.target.wants/; for i in *; do [ $i == \
systemd-tmpfiles-setup.service ] || rm -f $i; done); \
rm -f /lib/systemd/system/multi-user.target.wants/*;\
rm -f /etc/systemd/system/*.wants/*;\
rm -f /lib/systemd/system/local-fs.target.wants/*; \
rm -f /lib/systemd/system/sockets.target.wants/*udev*; \
rm -f /lib/systemd/system/sockets.target.wants/*initctl*; \
rm -f /lib/systemd/system/basic.target.wants/*;\
rm -f /lib/systemd/system/anaconda.target.wants/*;
VOLUME [ "/sys/fs/cgroup" ]
CMD ["/usr/sbin/init"]
```

### build指令

```shell
# 你需要进入 Dockerfile 文件所在的目录
cd centos
docker  build --tag local/centos:7.systemd .
...

docker images
---------------------------
REPOSITORY                                TAG             IMAGE ID       CREATED          SIZE
htalker/centos-systemd                    7               5cdcbd3f7015   17 seconds ago   204MB
centos                                    7               eeb6ee3f44bd   2 years ago      204MB
```

### 测试验证
```shell
# 宿主机为Linux：/sys/fs/cgroup
# 宿主机为Windows：\\wsl.localhost\docker-desktop\sys\fs\cgroup
# 停止容器的运行
docker stop centos7
docker rm centos7
# 清除之前不在线的容器实例
docker container prune
docker run --rm -it -d --name centos7 --hostname centos --privileged local/centos:7.systemd
...

# 进入容器 shell
docker exec -it centos7 bash
[root@centos /]#

# 容器内执行如下
yum makecache

yum install net-tools openssh-server

systemctl restart sshd

ps -ef
------------------
UID        PID  PPID  C STIME TTY          TIME CMD
root         1     0  0 06:38 ?        00:00:00 /usr/sbin/init
root        18     1  0 06:38 ?        00:00:00 /usr/lib/systemd/systemd-journald
root        20     0  0 06:38 pts/1    00:00:00 bash
root       144     1  0 06:40 ?        00:00:00 /usr/sbin/sshd -D
root       145    20  0 06:40 pts/1    00:00:00 ps -ef

netstat -tunpl
------------------
Active Internet connections (only servers)
Proto Recv-Q Send-Q Local Address           Foreign Address         State       PID/Program name    
tcp        0      0 0.0.0.0:22              0.0.0.0:*               LISTEN      144/sshd            
tcp6       0      0 :::22                   :::*                    LISTEN      144/sshd
```
> 注意：`--privileged` 表明容器必需是以特权模式运行，这对于一般应用来说不是安全的使用方式。不过好处就是我们可以通过 `systemctl` 指令来管理我们的服务了。

## 3.2.其它方法
    除了参照官方建议的systemd镜像的制作方法以外，其实有更为简介的方案，就是直接使用官方镜像：
        docker run --rm -it -d --name centos7s --hostname centos7s --privileged centos:7 /usr/sbin/init
        
        # 进入容器
        docker exec -it centos7s bash
        
        # 发现systemctl工具依旧可用
        systemctl status --all
        
        ...